In [65]:
from datasets import load_dataset
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
import fasttext

In [64]:
dataset01 = load_dataset("poleval2019_cyberbullying", "task01")
dataset02 = load_dataset("poleval2019_cyberbullying", "task02")

Reusing dataset poleval2019_cyber_bullying (/home/arkadius/.cache/huggingface/datasets/poleval2019_cyber_bullying/task01/1.0.0/ce6060c56dae43c469bab309a7573b86299b0bcc2484e85cfe0ae70b5f770450)


  0%|          | 0/2 [00:00<?, ?it/s]

Reusing dataset poleval2019_cyber_bullying (/home/arkadius/.cache/huggingface/datasets/poleval2019_cyber_bullying/task02/1.0.0/ce6060c56dae43c469bab309a7573b86299b0bcc2484e85cfe0ae70b5f770450)


  0%|          | 0/2 [00:00<?, ?it/s]

In [44]:
train_dataset01 = dataset01['train']
test_dataset01 = dataset01['test']
train_dataset02 = dataset02['train']
test_dataset02 = dataset02['test']

In [16]:
train_dataset01[:20]

{'text': ['Dla mnie faworytem do tytułu będzie Cracovia. Zobaczymy, czy typ się sprawdzi.',
  '@anonymized_account @anonymized_account Brawo ty Daria kibic ma być na dobre i złe',
  '@anonymized_account @anonymized_account Super, polski premier składa kwiaty na grobach kolaborantów. Ale doczekaliśmy czasów.',
  '@anonymized_account @anonymized_account Musi. Innej drogi nie mamy.',
  'Odrzut natychmiastowy, kwaśna mina, mam problem',
  'Jaki on był fajny xdd pamiętam, że spóźniłam się na jego pierwsze zajęcia i to sporo i za karę kazał mi usiąść w pierwszej ławce XD',
  '@anonymized_account No nie ma u nas szczęścia 😉',
  '@anonymized_account Dawno kogoś tak wrednego nie widziałam xd',
  '@anonymized_account @anonymized_account Zaległości były, ale ważne czy były wezwania do zapłaty z których się klub nie wywiązał.',
  '@anonymized_account @anonymized_account @anonymized_account Gdzie jest @anonymized_account . Brudziński jesteś kłamcą i marnym kutasem @anonymized_account',
  '@anonymiz

In [15]:
train_dataset02[:20]

{'text': ['Dla mnie faworytem do tytułu będzie Cracovia. Zobaczymy, czy typ się sprawdzi.',
  '@anonymized_account @anonymized_account Brawo ty Daria kibic ma być na dobre i złe',
  '@anonymized_account @anonymized_account Super, polski premier składa kwiaty na grobach kolaborantów. Ale doczekaliśmy czasów.',
  '@anonymized_account @anonymized_account Musi. Innej drogi nie mamy.',
  'Odrzut natychmiastowy, kwaśna mina, mam problem',
  'Jaki on był fajny xdd pamiętam, że spóźniłam się na jego pierwsze zajęcia i to sporo i za karę kazał mi usiąść w pierwszej ławce XD',
  '@anonymized_account No nie ma u nas szczęścia 😉',
  '@anonymized_account Dawno kogoś tak wrednego nie widziałam xd',
  '@anonymized_account @anonymized_account Zaległości były, ale ważne czy były wezwania do zapłaty z których się klub nie wywiązał.',
  '@anonymized_account @anonymized_account @anonymized_account Gdzie jest @anonymized_account . Brudziński jesteś kłamcą i marnym kutasem @anonymized_account',
  '@anonymiz

## Classifiers

#### Baessian + TF-IDF

In [55]:
def tf_idf(train_dataset, test_dataset):    
    vectorizer = TfidfVectorizer()
    vectorizer.fit(train_dataset)
    return vectorizer.transform(train_dataset).toarray(), vectorizer.transform(test_dataset).toarray()

train_dataset01_tfidf, test_dataset01_tfidf = tf_idf(train_dataset01['text'], test_dataset01['text'])
train_dataset02_tfidf, test_dataset02_tfidf = tf_idf(train_dataset02['text'], test_dataset02['text'])

In [56]:
gnb01 = GaussianNB()
gnb01.fit(train_dataset01_tfidf, train_dataset01['label'])

GaussianNB()

In [58]:
accuracy_score(gnb01.predict(test_dataset01_tfidf), test_dataset01['label'])

0.782

In [59]:
gnb02 = GaussianNB()
gnb02.fit(train_dataset02_tfidf, train_dataset02['label'])

GaussianNB()

In [60]:
accuracy_score(gnb02.predict(test_dataset02_tfidf), test_dataset02['label'])

0.787

#### Fasttext  

In [66]:
help(fasttext.FastText)

Help on module fasttext.FastText in fasttext:

NAME
    fasttext.FastText

DESCRIPTION
    # Copyright (c) 2017-present, Facebook, Inc.
    # All rights reserved.
    #
    # This source code is licensed under the MIT license found in the
    # LICENSE file in the root directory of this source tree.

FUNCTIONS
    cbow(*kargs, **kwargs)
    
    load_model(path)
        Load a model given a filepath and return a model object.
    
    read_args(arg_list, arg_dict, arg_names, default_values)
    
    skipgram(*kargs, **kwargs)
    
    supervised(*kargs, **kwargs)
    
    tokenize(text)
        Given a string of text, tokenize it and return a list of tokens
    
    train_supervised(*kargs, **kwargs)
        Train a supervised model and return a model object.
        
        input must be a filepath. The input text does not need to be tokenized
        as per the tokenize function, but it must be preprocessed and encoded
        as UTF-8. You might want to consult standard preprocessi

In [69]:
def to_fast_text_input_file(dataset, filename):
    with open(filename, "w") as f:
        for label, text in zip(dataset['label'], dataset['text']):
            f.write(f"__label__{label} {text}\n")

to_fast_text_input_file(train_dataset01, 'fasttext_train01.txt')
to_fast_text_input_file(test_dataset01, 'fasttext_test01.txt')
to_fast_text_input_file(train_dataset02, 'fasttext_train02.txt')
to_fast_text_input_file(test_dataset02, 'fasttext_test02.txt')

In [70]:
fasttext_model01 = fasttext.train_supervised('fasttext_train01.txt')
fasttext_model02 = fasttext.train_supervised('fasttext_train02.txt')

In [71]:
fasttext_model01.test('fasttext_test01.txt')

(1000, 0.872, 0.872)

In [73]:
fasttext_model02.test('fasttext_test02.txt')

(1000, 0.867, 0.867)

In [74]:
### transformers